# NEPA Delays Data Subsetting

## Subsetting and merging the PALS and FACTS datasets.

Within this project there are three datasets that are merged together. 

1. The FACTS dataset (commonattribute_c20220726.csv)
2. The PALS dataset (pals_ongoing_projects_11-2022.csv)
3. A dataset with calculated project areas and project diversity metrics (shannonindex_0120_v02.csv)

The FACTS dataset comes from the commonattributes geodatbase last accessed from https://data.fs.usda.gov/geodata/edw/datasets.php in July 2022. Our version was updated on July 13, 2022. The current version available for download was updated Sept 3, 2022. We initially import a .csv file of the common attributes data. The commonattribute_c20220726.csv was generated by exporting the Actv_CommonAttribute_PL layer from the Actv_CommonAttribute_PL.gdb as a .csv file from QGIS. 

The PALS dataset is from the University of Minnesota https://conservancy.umn.edu/handle/11299/211669. Here I use an updated version of the data from November 2022. Please email the first author for this dataset.

The dataset with the calculated project areas and project diversity metrics is the shannonindex_0120_v02.csv. This file was generated in R from the Actv_CommonAttribute_PL.gdb. See the R script in this repository. 


## 1. Load the appropriate libraries and read in the csv files

In [1]:
# Import important libraries/pachages
import pandas as pd #For working with dataframes
import numpy as np #For working with arrays
import datetime as dt #For converting dates to serial datetimes

# Set some display options
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 150)

# Set directories
raw_data_dir = '/Users/kathrynmurenbeeld/CODING/FOREST-TREATMENTS/DATA/FOREST-TREATMENTS/RAW/'
mod_data_dir = '/Users/kathrynmurenbeeld/CODING/FOREST-TREATMENTS/DATA/FOREST-TREATMENTS/MOD/'

# Set warnings
import warnings
warnings.filterwarnings(action='once')

In [2]:
# Load in the Common Attributes dataset

df = pd.read_csv(raw_data_dir + 'commonattribute_c20220726.csv')
df.head(2)

/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/2513058354.py:3: DtypeWarning: Columns (2,5,28,38,48,50,54,69,70,75,80,84,85,86,103,104) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(raw_data_dir + 'commonattribute_c20220726.csv')


,OBJECTID,SU_SECURITY_ID,AU_ORG,SUID,FACTS_ID,SUBUNIT,NAME,FEATURE_TYPE,SITE_NBR_OF_UNITS,UOM,ACTIVITY_CODE,ACTIVITY,LOCAL_QUALIFIER,METHOD_CODE,METHOD,EQUIPMENT_CODE,EQUIPMENT,FUND_CODES,COST_PER_UNIT,WORKFORCE_CODE,FISCAL_YEAR_PLANNED,FISCAL_YEAR_AWARDED,DATE_AWARDED,FISCAL_YEAR_COMPLETED,DATE_COMPLETED,NBR_UNITS_PLANNED,NBR_UNITS_ACCOMPLISHED,EXCLUDE_ACCOMPLISHMENT,TREATMENT_NAME,FUELS_KEYPOINT_AREA,ISWUI,FIREREGIME,CWPP,BIOMASS_UTILIZATION,NFPORS_CATEGORY,NFPORS_TREATMENT,PURPOSE_CODE,SALE_NAME,SALE_NUMBER,SALE_CATEGORY,UNIT_ID,PURCHASER_NAME,CONTRACT_PLANNED_TERM,AWARD_DATE,SALE_CLOSURE_DATE,BASE_YEAR,KV_NBR_UNITS_FUNDED,PERCENT_FUNDED,NEEDS,CAUSAL_AGENT,REFORESTATION_STATUS,EXAM_NBR,NEEDS_ADJUSTMENT,TSI,EVENT_NAME,EVENT_YEAR,IMPLEMENTATION_PROJECT,IMPL_PROJECT_NBR,IMPL_PROJECT_TYPE,NEPA_DOC_NBR,NEPA_DOC_TYPE,NEPA_PROJECT_NAME,NEPA_HFI,NEPA_HFRA,NEPA_SIGNED_DATE,ADMIN_REGION,ADMIN_FOREST,ADMIN_DISTRICT,AU_REGION,AU_FOREST,AU_DISTRICT,PROC_FOREST,OWNERSHIP,STATE_ABBR,PRODUCTIVITY_CLASS,LAND_SUITABILITY_CODE,COUNTY_NAME,CONG_DIST_NAME,LATITUDE,LONGITUDE,LEGAL_LOCATION,ASPECT,SLOPE,ELEVATION,WATERSHED_CODE,MGT_AREA_CODE,MGT_PRESCRIPTION_CODE,ACTIVITY_SITE_REMARKS,ACTIVITY_REMARKS,SU_CREATED_BY,SU_CREATED_DATE,SU_MODIFIED_BY,SU_MODIFIED_DATE,ACT_CREATED_BY,ACT_CREATED_DATE,ACT_MODIFIED_BY,ACT_MODIFIED_DATE,ACTIVITY_UNIT_CN,LU_CN,SUID_CN,EVENT_CN,NEPA_PROJECT_CN,PALS_PROJECT_CN,SALE_CN,IMPLEMENTATION_PROJECT_CN,UKCN,FS_UNIT_ID,CRC_VALUE,SHAPE_Length,SHAPE_Area
0,1,100420,100420,1004202000400042000,2000400042,000,"MACLEOD HARBOR #4 4-12-2,3",A,19.6,ACRES,4113,Stand Clearcut (EA/RH/FH),NaN,420.0,Tractor Logging,424.0,Ground Base Skidder *,XXXX,0.0,NaN,1959,1960.0,1959/11/18 00:00:00,1960.0,1959/11/18 00:00:00,19.6,19.6,N,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NOT REQD,NR,DEFAULT FOR NOT REQUIRED,N,N,NaN,10,4,20,10,4,20,1004.0,FS,AK,9.0,650,Valdez-Cordova,1ST CONGRESSIONAL DISTRICT OF ALASKA,NaN,NaN,"Bly Sound D1&2 T3S, R10E, sec 28",NaN,NaN,NaN,190202020004,NaN,NaN,Also called Macleod Harbor 320. TS records say...,TS records say some cut in 1952 by Soundview L...,SKESTI,2006/08/22 13:59:43,NaN,NaN,SKESTI,2006/08/22 14:00:59,FSDBA,2013/07/12 23:13:05,AU191881010602,L191881010602000,274826010602,AC1174086010602,1.010270e+11,1.010270e+11,NaN,NaN,100420_AC1174086010602,1004,9B87612A8A11E95C,0.022132,0.000013
1,2,100430,100430,1004302160300021000,2160300021,000,PATCH 14,A,5.8,ACRES,6130,Wildlife Habitat Create openings,NaN,200.0,Mechanical,235.0,Masticator,"CONT,NFWF",1334.0,CT,2012,2012.0,2012/06/01 00:00:00,NaN,NaN,5.8,5.8,N,NaN,NaN,Y,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,26864,DN,(PALS)DECISION NOTICE AND FONSI FOR THE BEAN N...,NaN,NaN,2012/02/14 00:00:00,10,4,30,10,4,30,1004.0,FS,AK,5.0,900,Kenai Peninsula,1ST CONGRESSIONAL DISTRICT OF ALASKA,60.5,-149.89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JPFORD,2012/09/20 19:05:33,JPFORD,2012/09/20 19:05:34,JPFORD,2012/09/25 15:57:55,JPFORD,2012/09/26 20:29:05,3719143010602,3719144010602,854818010602,3694001010602,3.411300e+04,5.366900e+04,NaN,NaN,100430_3694001010602,1004,A3AADC2987E666D0,0.009431,0.000004


In [3]:
# Convert dates from strings to datetime

df['NEPA_SIGNED_DATE'] = pd.to_datetime(df['NEPA_SIGNED_DATE'])
df['DATE_AWARDED'] = pd.to_datetime(df['DATE_AWARDED'])
df['DATE_COMPLETED'] = pd.to_datetime(df['DATE_COMPLETED'])

In [4]:
# Load in the PALS dataset and convert dates from strings to datetime

df_pals = pd.read_csv(mod_data_dir + 'pals_ongoing_projects_11-2022.csv', sep=";").reset_index()

df_pals = df_pals.drop(['index', 'Unnamed: 0'], axis=1)
df_pals['INITIATION DATE'] = pd.to_datetime(df_pals['INITIATION DATE'])
df_pals['DECISION SIGNED'] = pd.to_datetime(df_pals['DECISION SIGNED'])
df_pals['PROJECT NUMBER'] = df_pals['PROJECT NUMBER'].astype(str).replace('\.0', '', regex=True)
df_pals.head(2)

<>:9: DeprecationWarning: invalid escape sequence \.


,FOREST_ID,PROJECT NUMBER,PROJECT NAME,LMU (ACTUAL),LMU – REGION,LMU – FOREST,LMU – DISTRICT,PROJECT STATUS,PROJECT CREATED,CREATED FY,DECISION ID,DECISION NAME,INITIATION DATE,INITIATION FY,DECISION SIGNED,SIGNED FY,SIGNER LAST NAME,SIGNER FIRST NAME,SIGNER TITLE,DECISION TYPE,DECISION APPEAL RULE,PROJECT NOTICE AND COMMENT REGULATION,APPEALED OR OBJECTED?,NO COMMENTS OR ONLY SUPPORT?,LITIGATED?,FC Facility management – purpose,FR Research – purpose,HF Fuels management – purpose,HR Heritage resource management – purpose,LM Land ownership management – purpose,LW Land acquisition – purpose,MG Minerals and geology – purpose,PN Land management planning – purpose,RD Road management – purpose,RG Grazing management – purpose,"RO Regulations, directives, orders – purpose",RU Special area management – purpose,RW Recreation management – purpose,SU Special use management – purpose,TM Forest products – purpose,VM Vegetation management (non-forest products) – purpose,"WF Wildlife, fish, rare plants – purpose",WM Water management – purpose,AL Land use adjustments – activity,BL Boundary adjustments – activity,BM Biomass – activity,CP Plan creation/revision – activity,DC Directive creation/modification – activity,DR Road decommissioning – activity,DS Developed site management – activity,EC Environmental compliance actions – activity,ET Electric transmission – activity,"FACTS Activity (0,1)",FI Facility improvements/construction – activity,FN Fuel treatments – activity,FV Forest vegetation improvements – activity,GA Dispersed recreation management – activity,GP Grazing allotment management – activity,GR Grazing authorizations – activity,GT Geothermal – activity,HI Species habitat improvements – activity,HP Hydropower – activity,HR Heritage resource management – activity,LA Special use authorizations – activity,LP Land purchases – activity,MF Facility maintenance – activity,ML Abandoned mine land clean-up – activity,MO Minerals or geology plans of operations – activity,MP Plan amendment – activity,MT Trail management – activity,NC Special products sales – activity,NG Natural gas – activity,NW Noxious weed treatments – activity,OC Order creation/modification – activity,OL Oil – activity,PE Species population enhancements – activity,PJ Land exchanges – activity,RA Roadless area management – activity,RC Regulation creation/modification – activity,RD Road maintenance – activity,RE Research and development – activity,RI Road improvements/construction – activity,RV Rangeland vegetation improvements – activity,SA Special area management – activity,SC Scenery management – activity,SI Grazing structural improvements – activity,SL Solar – activity,SS Timber sales (salvage) – activity,TR Travel management – activity,TS Timber sales (green) – activity,WC Watershed improvements – activity,WD Wilderness management – activity,WI Wind – activity,UNIQUE PROJECT?,ELAPSED DAYS,UNIQUE DECISION?,ongoing,DISTRICT_ID,REGION_ID,DECISION_LEVEL,RegionName,forest,calendarYearSigned,calendarYearInitiated
0,0.0,21846,National Forest System Land Management Plannin...,All Districts-level Units (11000000),AllUnit,AllUnit,All Districts-level Units (11000000),Complete,08/02/2007,2007.0,20023.0,National Forest System Land Management Plannin...,2007-05-11,2007.0,2008-04-09,2008.0,Rey,Mark,Secretary of Agriculture,ROD,None,Not Subject to Regulations,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,334.0,NaN,NaN,0.0,0.0,Region/HQ,AllUnit,NaN,2008.0,2007.0
1,0.0,35570,Gypsy Moth Management in the United States: A ...,All Districts-level Units (11000000),AllUnit,AllUnit,All Districts-level Units (11000000),Complete,03/28/2011,2011.0,36915.0,Gypsy Moth Supplemental Environmental Impact S...,2004-04-29,2004.0,2012-11-28,2013.0,Hubbard,James,Deputy Ch

In [5]:
# Rename the PROJECT NUMBER column to NEPA_DOC_NBR 
df_pals = df_pals.rename(columns={"PROJECT NUMBER": "NEPA_DOC_NBR"})

## 2. Combine the PALS dataset with the Shannon Index csv before filtering by year

In [6]:
shann = pd.read_csv(raw_data_dir + 'shannonindex_0120_v02.csv')
shann = shann.drop('Unnamed: 0', axis=1)

In [7]:
# Check the number of unique projects in each dataset
print(len(df['NEPA_DOC_NBR'].unique()))
print(len(df_pals['NEPA_DOC_NBR'].unique()))
print(len(shann['NEPA_DOC_NBR'].unique()))


18158
40721
4852


In [8]:
#Merge the PALS dataset with the Shannon Index dataset
pals_diversity = pd.merge(df_pals, shann, how="left", on='NEPA_DOC_NBR')


### Important Note!!
**df_na** - this dataframe is what we will merge to the FACTS data later to make sure we account for all the projects with a calculated project area ("footprint").

In [9]:
# df_na: this is what we will merge to the PALS data later to make sure 
# we account for all the projects with a calculated footprint.

df_na = pals_diversity[~pals_diversity['numact'].isna()] 
df_notna = pals_diversity[pals_diversity['numact'].isna()]
print(len(df_na))
print(len(df_notna))

4823
35898


#### Check some of the missing values between the PALS and the Shannon datasets

In [10]:
facts_projs_in_pals = list(df_na['NEPA_DOC_NBR'].unique())
facts_projs = list(shann['NEPA_DOC_NBR'].unique())
missing_facts_projs = list(set(facts_projs) - set(facts_projs_in_pals))


In [13]:
print("There are " + str(len(missing_facts_projs)) + " projects in the Shannon dataset that are not in the PALS dataset")


There are 29 projects in the Shannon dataset that are not in the PALS dataset


## 3. Filter the FACTS dataset by NEPA_SIGNED_DATE >= '2009-01-01'

Because we will eventually filter the PALS data by NEPA_INIT >= '2009-01-01', we can subset the FACTS dataset to help the next few steps go a bit quicker. Here we assume that projects signed on Jan 1, 2009 were initiated by Jan 1, 2009. The projects in the Shannon Index were also taken from the FACTS dataset using this filter.

In the original testing of this subset process, I filtered on NEPA_SIGNED_DATE >= '2007-01-01' hence the dataframe is named df_07

In [14]:
# Filter the FACTS dataset for projects signed on or after Jan 1, 2009

df_07 = df.loc[df['NEPA_SIGNED_DATE'] >= '2009-01-01 00:00:00'].copy().reset_index()
df_07 = df_07.drop(['index'], axis=1)


In [15]:
df_07.head(2)

,OBJECTID,SU_SECURITY_ID,AU_ORG,SUID,FACTS_ID,SUBUNIT,NAME,FEATURE_TYPE,SITE_NBR_OF_UNITS,UOM,ACTIVITY_CODE,ACTIVITY,LOCAL_QUALIFIER,METHOD_CODE,METHOD,EQUIPMENT_CODE,EQUIPMENT,FUND_CODES,COST_PER_UNIT,WORKFORCE_CODE,FISCAL_YEAR_PLANNED,FISCAL_YEAR_AWARDED,DATE_AWARDED,FISCAL_YEAR_COMPLETED,DATE_COMPLETED,NBR_UNITS_PLANNED,NBR_UNITS_ACCOMPLISHED,EXCLUDE_ACCOMPLISHMENT,TREATMENT_NAME,FUELS_KEYPOINT_AREA,ISWUI,FIREREGIME,CWPP,BIOMASS_UTILIZATION,NFPORS_CATEGORY,NFPORS_TREATMENT,PURPOSE_CODE,SALE_NAME,SALE_NUMBER,SALE_CATEGORY,UNIT_ID,PURCHASER_NAME,CONTRACT_PLANNED_TERM,AWARD_DATE,SALE_CLOSURE_DATE,BASE_YEAR,KV_NBR_UNITS_FUNDED,PERCENT_FUNDED,NEEDS,CAUSAL_AGENT,REFORESTATION_STATUS,EXAM_NBR,NEEDS_ADJUSTMENT,TSI,EVENT_NAME,EVENT_YEAR,IMPLEMENTATION_PROJECT,IMPL_PROJECT_NBR,IMPL_PROJECT_TYPE,NEPA_DOC_NBR,NEPA_DOC_TYPE,NEPA_PROJECT_NAME,NEPA_HFI,NEPA_HFRA,NEPA_SIGNED_DATE,ADMIN_REGION,ADMIN_FOREST,ADMIN_DISTRICT,AU_REGION,AU_FOREST,AU_DISTRICT,PROC_FOREST,OWNERSHIP,STATE_ABBR,PRODUCTIVITY_CLASS,LAND_SUITABILITY_CODE,COUNTY_NAME,CONG_DIST_NAME,LATITUDE,LONGITUDE,LEGAL_LOCATION,ASPECT,SLOPE,ELEVATION,WATERSHED_CODE,MGT_AREA_CODE,MGT_PRESCRIPTION_CODE,ACTIVITY_SITE_REMARKS,ACTIVITY_REMARKS,SU_CREATED_BY,SU_CREATED_DATE,SU_MODIFIED_BY,SU_MODIFIED_DATE,ACT_CREATED_BY,ACT_CREATED_DATE,ACT_MODIFIED_BY,ACT_MODIFIED_DATE,ACTIVITY_UNIT_CN,LU_CN,SUID_CN,EVENT_CN,NEPA_PROJECT_CN,PALS_PROJECT_CN,SALE_CN,IMPLEMENTATION_PROJECT_CN,UKCN,FS_UNIT_ID,CRC_VALUE,SHAPE_Length,SHAPE_Area
0,2,100430,100430,1004302160300021000,2160300021,000,PATCH 14,A,5.8,ACRES,6130,Wildlife Habitat Create openings,NaN,200.0,Mechanical,235.0,Masticator,"CONT,NFWF",1334.0,CT,2012,2012.0,2012-06-01,NaN,NaT,5.8,5.8,N,NaN,NaN,Y,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,26864,DN,(PALS)DECISION NOTICE AND FONSI FOR THE BEAN N...,NaN,NaN,2012-02-14,10,4,30,10,4,30,1004.0,FS,AK,5.0,900,Kenai Peninsula,1ST CONGRESSIONAL DISTRICT OF ALASKA,60.5,-149.89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JPFORD,2012/09/20 19:05:33,JPFORD,2012/09/20 19:05:34,JPFORD,2012/09/25 15:57:55,JPFORD,2012/09/26 20:29:05,3719143010602,3719144010602,854818010602,3694001010602,34113.0,53669.0,NaN,NaN,100430_3694001010602,1004,A3AADC2987E666D0,0.009431,0.000004
1,3,100430,100430,1004302160300021000,2160300021,000,PATCH 14,A,5.8,ACRES,4341,Stocking Survey,NaN,900.0,Survey Methods,0.0,NaN,NFVW,0.0,NaN,2016,2016.0,2016-06-25,2016.0,2016-06-25,5.8,5.8,N,NaN,NaN,Y,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,26864,DN,(PALS)DECISION NOTICE AND FONSI FOR THE BEAN N...,NaN,NaN,2012-02-14,10,4,30,10,4,30,1004.0,FS,AK,5.0,900,Kenai Peninsula,1ST CONGRESSIONAL DISTRICT OF ALASKA,60.5,-149.89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JPFORD,2012/09/20 19:05:33,JPFORD,2012/09/20 19:05:34,JPFORD,2012/09/25 15:57:57,RHAMMER,2019/08/16 16:08:45,3719143010602,3719144010602,854818010602,3694009010602,34113.0,53669.0,NaN,NaN,100430_3694009010602,1004,A3AADC29E5B78415,0.009431,0.000004


In [16]:
print(df_07['DATE_COMPLETED'].min())
print(df_07['DATE_COMPLETED'].max())
print(df_07['NEPA_SIGNED_DATE'].min())
print(df_07['NEPA_SIGNED_DATE'].max())
print(len(df_07['NEPA_DOC_NBR'].unique()))

1943-01-01 00:00:00
2023-07-26 00:00:00
2009-01-02 00:00:00
2100-01-01 00:00:00
5464


In [17]:
# Are there any projects that are missing between the shannon index and the FACTS >=2009-01-01?
facts_projs_commatt = list(df_07['NEPA_DOC_NBR'].unique())

missing_facts_projs2 = list(set(facts_projs) - set(facts_projs_commatt))

In [18]:
missing_facts_projs2

[]

The project IDs match between the FACTS dataset and the shannon index.

Compare again after removing activities or contracts that are recorded as being completed or awarded pre-NEPA signed dates.

## 4. Add variables, calculate time variables, calculate completness, calculate diversity and richness

We break out the general activity types based on the activity codes in the FACTS dataset. The code groupings used by the Forest Service can be found in Appendix B: FACTS Activity Codes https://www.fs.usda.gov/Internet/FSE_DOCUMENTS/fseprd539041.pdf.  

Here RICHNESS will eventually be populated with a count of the unique activity types for each project.


In [19]:
## First, add the dummy columns for activity type
## Also add empty columns for RICHNESS and INTENSITY. These will be filled in later. 

df_07['FIRE'] = np.where((df_07['ACTIVITY_CODE'] >= 1000) & (df_07['ACTIVITY_CODE'] < 2000), 
                                  1, 0)
df_07['RANGE'] = np.where((df_07['ACTIVITY_CODE'] >= 2000) & (df_07['ACTIVITY_CODE'] < 3000),
                                  1, 0)
df_07['REC'] = np.where((df_07['ACTIVITY_CODE'] >= 3000) & (df_07['ACTIVITY_CODE'] < 4000),
                                  1, 0)
df_07['TIMBER'] = np.where((df_07['ACTIVITY_CODE'] >= 4000) & (df_07['ACTIVITY_CODE'] < 5000),
                                  1, 0)
df_07['SAW'] = np.where((df_07['ACTIVITY_CODE'] >= 5000) & (df_07['ACTIVITY_CODE'] < 6000),
                                  1, 0)
df_07['WILDLIFE'] = np.where((df_07['ACTIVITY_CODE'] >= 6000) & (df_07['ACTIVITY_CODE'] < 7000),
                                  1, 0)
df_07['RESTORE'] = np.where((df_07['ACTIVITY_CODE'] >= 7000) & (df_07['ACTIVITY_CODE'] < 8000),
                                  1, 0)
df_07['MISC'] = np.where((df_07['ACTIVITY_CODE'] >= 8000) & (df_07['ACTIVITY_CODE'] < 9000),
                                  1, 0)
df_07['ENGI'] = np.where((df_07['ACTIVITY_CODE'] >= 9000) & (df_07['ACTIVITY_CODE'] < 10000),
                                  1, 0)
df_07['RICHNESS'] = 0

#df_07['INTENSITY'] = 0

df_07.head(2)


,OBJECTID,SU_SECURITY_ID,AU_ORG,SUID,FACTS_ID,SUBUNIT,NAME,FEATURE_TYPE,SITE_NBR_OF_UNITS,UOM,ACTIVITY_CODE,ACTIVITY,LOCAL_QUALIFIER,METHOD_CODE,METHOD,EQUIPMENT_CODE,EQUIPMENT,FUND_CODES,COST_PER_UNIT,WORKFORCE_CODE,FISCAL_YEAR_PLANNED,FISCAL_YEAR_AWARDED,DATE_AWARDED,FISCAL_YEAR_COMPLETED,DATE_COMPLETED,NBR_UNITS_PLANNED,NBR_UNITS_ACCOMPLISHED,EXCLUDE_ACCOMPLISHMENT,TREATMENT_NAME,FUELS_KEYPOINT_AREA,ISWUI,FIREREGIME,CWPP,BIOMASS_UTILIZATION,NFPORS_CATEGORY,NFPORS_TREATMENT,PURPOSE_CODE,SALE_NAME,SALE_NUMBER,SALE_CATEGORY,UNIT_ID,PURCHASER_NAME,CONTRACT_PLANNED_TERM,AWARD_DATE,SALE_CLOSURE_DATE,BASE_YEAR,KV_NBR_UNITS_FUNDED,PERCENT_FUNDED,NEEDS,CAUSAL_AGENT,REFORESTATION_STATUS,EXAM_NBR,NEEDS_ADJUSTMENT,TSI,EVENT_NAME,EVENT_YEAR,IMPLEMENTATION_PROJECT,IMPL_PROJECT_NBR,IMPL_PROJECT_TYPE,NEPA_DOC_NBR,NEPA_DOC_TYPE,NEPA_PROJECT_NAME,NEPA_HFI,NEPA_HFRA,NEPA_SIGNED_DATE,ADMIN_REGION,ADMIN_FOREST,ADMIN_DISTRICT,AU_REGION,AU_FOREST,AU_DISTRICT,PROC_FOREST,OWNERSHIP,STATE_ABBR,PRODUCTIVITY_CLASS,LAND_SUITABILITY_CODE,COUNTY_NAME,CONG_DIST_NAME,LATITUDE,LONGITUDE,LEGAL_LOCATION,ASPECT,SLOPE,ELEVATION,WATERSHED_CODE,MGT_AREA_CODE,MGT_PRESCRIPTION_CODE,ACTIVITY_SITE_REMARKS,ACTIVITY_REMARKS,SU_CREATED_BY,SU_CREATED_DATE,SU_MODIFIED_BY,SU_MODIFIED_DATE,ACT_CREATED_BY,ACT_CREATED_DATE,ACT_MODIFIED_BY,ACT_MODIFIED_DATE,ACTIVITY_UNIT_CN,LU_CN,SUID_CN,EVENT_CN,NEPA_PROJECT_CN,PALS_PROJECT_CN,SALE_CN,IMPLEMENTATION_PROJECT_CN,UKCN,FS_UNIT_ID,CRC_VALUE,SHAPE_Length,SHAPE_Area,FIRE,RANGE,REC,TIMBER,SAW,WILDLIFE,RESTORE,MISC,ENGI,RICHNESS,INTENSITY
0,2,100430,100430,1004302160300021000,2160300021,000,PATCH 14,A,5.8,ACRES,6130,Wildlife Habitat Create openings,NaN,200.0,Mechanical,235.0,Masticator,"CONT,NFWF",1334.0,CT,2012,2012.0,2012-06-01,NaN,NaT,5.8,5.8,N,NaN,NaN,Y,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,26864,DN,(PALS)DECISION NOTICE AND FONSI FOR THE BEAN N...,NaN,NaN,2012-02-14,10,4,30,10,4,30,1004.0,FS,AK,5.0,900,Kenai Peninsula,1ST CONGRESSIONAL DISTRICT OF ALASKA,60.5,-149.89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JPFORD,2012/09/20 19:05:33,JPFORD,2012/09/20 19:05:34,JPFORD,2012/09/25 15:57:55,JPFORD,2012/09/26 20:29:05,3719143010602,3719144010602,854818010602,3694001010602,34113.0,53669.0,NaN,NaN,100430_3694001010602,1004,A3AADC2987E666D0,0.009431,0.000004,0,0,0,0,0,1,0,0,0,0,0
1,3,100430,100430,1004302160300021000,2160300021,000,PATCH 14,A,5.8,ACRES,4341,Stocking Survey,NaN,900.0,Survey Methods,0.0,NaN,NFVW,0.0,NaN,2016,2016.0,2016-06-25,2016.0,2016-06-25,5.8,5.8,N,NaN,NaN,Y,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,26864,DN,(PALS)DECISION NOTICE AND FONSI FOR THE BEAN N...,NaN,NaN,2012-02-14,10,4,30,10,4,30,1004.0,FS,AK,5.0,900,Kenai Peninsula,1ST CONGRESSIONAL DISTRICT OF ALASKA,60.5,-149.89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JPFORD,2012/09/20 19:05:33,JPFORD,2012/09/20 19:05:34,JPFORD,2012/09/25 15:57:57,RHAMMER,2019/08/16 16:08:45,3719143010602,3719144010602,854818010602,3694009010602,34113.0,53669.0,NaN,NaN,100430_3694009010602,1004,A3AADC29E5B78415,0.009431,0.000004,0,0,0,1,0,0,0,0,0,0,0


In [20]:
# Calculate the difference between planned area treated and actual area treated

df_07['ACT_AREA_DIFF'] = df_07['NBR_UNITS_ACCOMPLISHED'] - df_07['NBR_UNITS_PLANNED']


In [21]:
# Create a dummy variable for whether an activity was completed or not

df_07['act_comp'] = df_07['FISCAL_YEAR_COMPLETED'].fillna(0)
#df_comm_dropdups['act_comp'] = df_comm_dropdups['NBR_UNITS_ACCOMPLISHED'].astype('float64')
df_07['act_comp'].dtype

dtype('float64')

In [22]:
# Replace the dummy variable with a better name and drop act_comp
comp_conditions = [ (df_07['act_comp'] == 0), (df_07['act_comp'] > 0)]
comp_choices = [0, 1]

df_07['ACT_COMPLETED'] = np.select(comp_conditions, comp_choices, default=0)
df_07 = df_07.drop(['act_comp'], axis=1)

In [23]:
# Calculate the years from when an activity was planned to occur and when the activity was either awarded 
# a contract or completed.

df_07['PLAN_TO_AWARD'] = df_07['FISCAL_YEAR_AWARDED'] - df_07['FISCAL_YEAR_PLANNED']

df_07['PLAN_TO_COMPLETE'] = df_07['FISCAL_YEAR_COMPLETED'] - df_07['FISCAL_YEAR_PLANNED']

## 5. Aggregate the dataset by unique NEPA project number.

### 5.1 Before Aggregating the Dataset and calculating diversity, richness, and evenness, calculate the NEPA to first activity and NEPA to contract
We will removed activities that occurred before NEPA was signed. 

Then remove those activities that occurred before NEPA was signed. 

Check this against the projects in the shannon index dataset.

In [24]:
df_07['NEPA_TO_FA'] = df_07['DATE_COMPLETED'] - df_07['NEPA_SIGNED_DATE']
df_07['NEPA_TO_CTRT'] = df_07['DATE_AWARDED'] - df_07['NEPA_SIGNED_DATE']

df_07['NEPA_TO_FA'] = pd.to_numeric(df_07['NEPA_TO_FA'].dt.days, downcast='integer')
df_07['NEPA_TO_CTRT'] = pd.to_numeric(df_07['NEPA_TO_CTRT'].dt.days, downcast='integer')

In [25]:
# Remove the activities that are recorded as being completed prior to NEPA signing 
# Remove the activities that are recorded as being awarded a contract prior to NEPA signing

df_07_noneg = df_07.loc[(df_07['NEPA_TO_FA'] >= 0) | (df_07['NEPA_TO_FA'].isna())].copy()
df_07_noneg = df_07_noneg.loc[(df_07_noneg['NEPA_TO_CTRT'] >= 0) | (df_07_noneg['NEPA_TO_CTRT'].isna())].copy()


In [26]:
# Check that the activities were removed. 

#df_07_noneg.loc[df_07_noneg['NEPA_TO_FA'] < 0]
df_07_noneg.loc[df_07_noneg['NEPA_TO_CTRT'] < 0]

,OBJECTID,SU_SECURITY_ID,AU_ORG,SUID,FACTS_ID,SUBUNIT,NAME,FEATURE_TYPE,SITE_NBR_OF_UNITS,UOM,ACTIVITY_CODE,ACTIVITY,LOCAL_QUALIFIER,METHOD_CODE,METHOD,EQUIPMENT_CODE,EQUIPMENT,FUND_CODES,COST_PER_UNIT,WORKFORCE_CODE,FISCAL_YEAR_PLANNED,FISCAL_YEAR_AWARDED,DATE_AWARDED,FISCAL_YEAR_COMPLETED,DATE_COMPLETED,NBR_UNITS_PLANNED,NBR_UNITS_ACCOMPLISHED,EXCLUDE_ACCOMPLISHMENT,TREATMENT_NAME,FUELS_KEYPOINT_AREA,ISWUI,FIREREGIME,CWPP,BIOMASS_UTILIZATION,NFPORS_CATEGORY,NFPORS_TREATMENT,PURPOSE_CODE,SALE_NAME,SALE_NUMBER,SALE_CATEGORY,UNIT_ID,PURCHASER_NAME,CONTRACT_PLANNED_TERM,AWARD_DATE,SALE_CLOSURE_DATE,BASE_YEAR,KV_NBR_UNITS_FUNDED,PERCENT_FUNDED,NEEDS,CAUSAL_AGENT,REFORESTATION_STATUS,EXAM_NBR,NEEDS_ADJUSTMENT,TSI,EVENT_NAME,EVENT_YEAR,IMPLEMENTATION_PROJECT,IMPL_PROJECT_NBR,IMPL_PROJECT_TYPE,NEPA_DOC_NBR,NEPA_DOC_TYPE,NEPA_PROJECT_NAME,NEPA_HFI,NEPA_HFRA,NEPA_SIGNED_DATE,ADMIN_REGION,ADMIN_FOREST,ADMIN_DISTRICT,AU_REGION,AU_FOREST,AU_DISTRICT,PROC_FOREST,OWNERSHIP,STATE_ABBR,PRODUCTIVITY_CLASS,LAND_SUITABILITY_CODE,COUNTY_NAME,CONG_DIST_NAME,LATITUDE,LONGITUDE,LEGAL_LOCATION,ASPECT,SLOPE,ELEVATION,WATERSHED_CODE,MGT_AREA_CODE,MGT_PRESCRIPTION_CODE,ACTIVITY_SITE_REMARKS,ACTIVITY_REMARKS,SU_CREATED_BY,SU_CREATED_DATE,SU_MODIFIED_BY,SU_MODIFIED_DATE,ACT_CREATED_BY,ACT_CREATED_DATE,ACT_MODIFIED_BY,ACT_MODIFIED_DATE,ACTIVITY_UNIT_CN,LU_CN,SUID_CN,EVENT_CN,NEPA_PROJECT_CN,PALS_PROJECT_CN,SALE_CN,IMPLEMENTATION_PROJECT_CN,UKCN,FS_UNIT_ID,CRC_VALUE,SHAPE_Length,SHAPE_Area,FIRE,RANGE,REC,TIMBER,SAW,WILDLIFE,RESTORE,MISC,ENGI,RICHNESS,INTENSITY,ACT_AREA_DIFF,ACT_COMPLETED,PLAN_TO_AWARD,PLAN_TO_COMPLETE,NEPA_TO_FA,NEPA_TO_CTRT


In [27]:
# Are there any projects that are missing between the shannon index and the FACTS >=2009-01-01 without 
# activities or contracts with pre-NEPA dates?
facts_projs_commatt_noneg = list(df_07_noneg['NEPA_DOC_NBR'].unique())

#missing_facts_projs3 = list(set(facts_projs_commatt) - set(facts_projs_commatt_noneg))
missing_facts_projs3 = list(set(facts_projs) - set(facts_projs_commatt_noneg)) # facts_proj = list of unique projects from df_na.
# which is from the shannon and simpson index merged with PALS

In [28]:
print("When activities or contracts that have pre-NEPA dates are removed there are " + str(len(missing_facts_projs3)) +
      " projects removed.")

When activities or contracts that have pre-NEPA dates are removed there are 36 projects removed.


### 5.2 Calculate Shannon Diveristy and Shannon Evenness Metrics

**Important** These values will not be used or referenced in the brief. The Shannon diveristy metrics referenced in the brief come from the shannonindex_v02.csv.

Only the **PROJ_RICHNESS** is used within the brief.


In [29]:
# Create functions to caluclate Shannon Diversity and Shannon Evenness

def shannon_diversity(n, N):
    """Calculates the Shannon Diversity"""
    """n is a list/array of the number of activities in each unique area or of each activity type"""
    """N is the total number of activities in the project"""
    
    area_div = -sum(((n/N) * np.log(n/N)))
    
    return area_div

In [30]:
def shannon_evenness(D, S):
    """Calculates the Shannon Evenness"""
    """D is Shannon Diversity calculated for the project"""
    """S is the total number of unique areas or activities in a project"""
    
    area_even = D/np.log(S)
    
    return area_even

In [31]:
# Create a list of the unique project numbers.

projects = list(df_07_noneg['NEPA_DOC_NBR'].unique())

In [32]:
# Calculate the diversity, evenness, and richness by unique activity areas.

# Create empty lists. area_projs will be a list of the project numbers
area_div = []
area_eve = []
area_rich = []
area_projs = []

for p in projects:
    n = np.array(df_07_noneg.loc[df_07_noneg['NEPA_DOC_NBR'] == p].groupby(['LATITUDE', 'LONGITUDE']).size())
    N = sum(n)
    S = len(n)
    area_rich.append(S)
    D = shannon_diversity(n,N)
    #print(D)
    area_div.append(D)
    E = shannon_evenness(D,S)
    area_eve.append(E)
    area_projs.append(p)
    #print(E)

/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: divide by zero encountered in log
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: divide by zero encountered in log
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: divide by zero encountered in log
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46

/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: divide by zero encountered in log
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: divide by zero encountered in log
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: divide by zero encountered in log
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: divide by zero encountered in log
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0g

/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: divide by zero encountered in log
  area_even = D/np.log(S)
/v

/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np

/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: divide by zero encountered in log
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: divide by zero encountered in log
  area_even = D/np.log(S)
/var/folders

...

/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: divide by zero encountered in log
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: divide by zero encountered in log
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders

/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: divide by zero encountered in log
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/v

/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: divide by zero encountered in log
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: divide by zero encountered in log
  area_even = D/np.log(S)
/var/folders

/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: divide by zero encountered in log
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: divide by zero encountered in log
  area_even = D/np.log(S)
/var/folders

/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np

In [33]:
print(len(area_div))
print(len(area_eve))
print(len(area_rich))
print(len(area_projs))

5419
5419
5419
5419


In [34]:
# Calculate the diversity, evenness, and richness by unique activity type.

# Create empty lists. area_projs will be a list of the project numbers

proj_div = []
proj_eve = []
proj_rich = []
proj_projs = []

for p in projects:
    n = np.array(df_07_noneg.loc[df_07_noneg['NEPA_DOC_NBR'] == p].groupby(['FIRE', 
                                                                            'RANGE', 
                                                                            'REC',
                                                                            'TIMBER',
                                                                            'SAW',
                                                                            'WILDLIFE',
                                                                            'RESTORE', 
                                                                            'MISC', 
                                                                            'ENGI']).size())
    #print(n)
    N = sum(n)
    S = len(n)
    proj_rich.append(S)
    D = shannon_diversity(n,N)
    #print(D)
    proj_div.append(D)
    #print(S)
    E = shannon_evenness(D,S)
    proj_eve.append(E)
    proj_projs.append(p)
    #print(E)
    

/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np

/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np

/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np

/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np

/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np

...

/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np

/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np

/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np

/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np

/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np.log(S)
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/1203830900.py:6: RuntimeWarning: invalid value encountered in double_scalars
  area_even = D/np

In [35]:
print(len(proj_div))
print(len(proj_eve))
print(len(proj_rich))
print(len(proj_projs))


5419
5419
5419
5419


In [36]:
# Create an empty dataframe for the diversity metrics. 
# We will fill in the dataframe with the lists created of project or area diversity.

df_div_mets = pd.DataFrame() 


In [37]:
# Fill in the df_div_mets dataframe.

df_div_mets['NEPA_DOC_NBR'] = np.array(proj_projs)
df_div_mets['PROJ_RICHNESS'] = np.array(proj_rich)
df_div_mets['PROJ_DIVERSE'] = np.array(proj_div)
df_div_mets['PROJ_EVENNESS'] = np.array(proj_eve)

df_div_mets['AREA_DOC_NBR'] = np.array(area_projs)
df_div_mets['AREA_RICHNESS'] = np.array(area_rich)
df_div_mets['AREA_DIVERSE'] = np.array(area_div)
df_div_mets['AREA_EVENNESS'] = np.array(area_eve)

In [38]:
# Check that the lists were brought into the dataframe correctly. 

df_div_mets.head(10)

,NEPA_DOC_NBR,PROJ_RICHNESS,PROJ_DIVERSE,PROJ_EVENNESS,AREA_DOC_NBR,AREA_RICHNESS,AREA_DIVERSE,AREA_EVENNESS
0,26864,3,0.915050,0.832914,26864,2,0.115774,0.167027
1,36819,1,-0.000000,NaN,36819,115,4.490938,0.946470
2,50709,1,-0.000000,NaN,50709,6,1.569153,0.875761
3,57561,1,-0.000000,NaN,57561,24,3.128280,0.984338
4,19676,1,-0.000000,NaN,19676,6,1.467293,0.818912
5,50639,1,-0.000000,NaN,50639,0,0.000000,-0.000000
6,38120,2,0.636514,0.918296,38120,9,1.859062,0.846096
7,48044,1,-0.000000,NaN,48044,5,1.575547,0.978942
8,47534,3,0.835275,0.760300,47534,21,3.003626,0.986567
9,56085,1,-0.000000,NaN,56085,1,-0.000000,NaN


In [39]:
# In order to properly merge this to the df_07 make sure to sort ascending by the NEPA_DOC_NBR

df_div_mets = df_div_mets.sort_values(by=['NEPA_DOC_NBR']).reset_index()
df_div_mets.head(10)

,index,NEPA_DOC_NBR,PROJ_RICHNESS,PROJ_DIVERSE,PROJ_EVENNESS,AREA_DOC_NBR,AREA_RICHNESS,AREA_DIVERSE,AREA_EVENNESS
0,4554,10031,3,0.582584,0.530291,10031,1,-0.000000,NaN
1,3888,10045,3,0.488871,0.444989,10045,7,0.620920,0.319090
2,2294,1006,3,0.836109,0.761059,1006,74,4.101769,0.952999
3,2131,10083,3,0.735189,0.669198,10083,34,3.388206,0.960822
4,3867,10130,4,1.064084,0.767575,10130,134,4.755605,0.970960
5,1398,10212,3,0.799846,0.728052,10212,74,4.232771,0.983436
6,962,10348,3,0.596580,0.543030,10348,2,0.081510,0.117595
7,1814,10383,1,-0.000000,NaN,10383,1,-0.000000,NaN
8,489,10457,4,0.544810,0.392997,10457,18,2.859266,0.989238
9,1244,10501,1,-0.000000,NaN,10501,9,2.163956,0.984859


### 5.3 Group the Common Attributes Dataset by Project Number

Here we finally aggregate the df_07 dataset by the unique project number. The following variables are created for each project:

* **NEPA_SIGNED_DATE** - the minimum NEPA_SIGNED_DATE
* **REGION** - the average ADMIN_REGION (sometimes projects occur in different regions. Once the FACTS data (df_07) is merged with the PALS, one can use the **Region** column in the PALS data.
* **DATE_COMP_MIN** - the minimum DATE_COMPLETED of all activities within a project
* **DATE_COMP_MAX** - the maximum DATE_COMPLETED of all activities within a project
* **PROJECT_DUR_CURRENT** -  the current project duration calculated by subtracting the minimum DATE_COMPELTED from the maximum DATE_COMPLETED
* **DATE_AWARD_MIN**, **DATE_AWARD_MAX** - the minimum or maximum DATE_AWARDED (the date an activity in a project was awarded a contract)
* **FY_AWARD_MIN** , **FY_AWARD_MAX** - the minimum or maximum FISCAL_YEAR_AWARDED of all activities within a project.
* **FY_PLAN_MIN**, **FY_PLAN_MAX** - the minimum or maximum FISCAL_YEAR_PLANNED of all activities within a project.
* **FY_COMP_MIN**, **FY_COMP_MAX** - the minimum or maximum FISCAL_YEAR_COMPLETED of all activities within a project.
* **PROJECT_DUR_PLAN_YR** - the planned project duration in years calculated by subtracting the minimum FISCAL_YEAR_PLANNED from the maximum FISCAL_YEAR_PLANNED.
* **ACT_AREA_DIFF** - the average difference in activities area/size as planned to completed (accomplished)
* **TOT_AREA_PLAN**, **TOT_AREA_COMP** - the sum of the total activity areas as planned (TOT_AREA_PLAN) and accomplished (TOT_AREA_COMP)
* **NBR_ACTIVITIES** - the sum of the total number of activities
* **FIRE**, **RANGE**, **REC**, **TIMBER**, **SAW**, **WILDLIFE**, **RESTORE**, **MISC**, **ENGI** - sum of the total number of activities in a project by type
* **FIRE_AVE_AREA**, **RANGE_AVE_AREA**, **REC_AVE_ARE**, **TIMBER_AVE_AREA**, **SAW_AVE_AREA**, **WILDLIFE_AVE_AREA**, **RESTORE_AVE_AREA**, **MISC_AVE_AREA**, **ENGI_AVE_AREA** - the average size of each activity type in each project
* **RICHNESS_TEST** - sum of RICHNESS for each activity. Was created as a check and should match the **PROJ_RICHNESS** variable 
* **PERCENT_PROJ_COMP** - the average of the ACT_COMPLETE variable. ACT_COMPLETE is either 0 (DATE_COMPLETE is NaT) or 1 (DATE_COMPLETE has a date)
* **PLAN_TO_COMP_AVE** - the average time from FY_PLANNED for an activity and FY_COMP for an activity  
* **PLAN_TO_AWARD_AVE** - the average time from FY_PLANNED for an activity to FY_COMP for an activity
* **FIRST_ACTIVITY** - **NOTE** the activities were not sorted by date so this variable is incorrect
* **LAST_ACTIVITY** - **NOTE** the activities were not sorted by date so this variable is incorrect

In [40]:
# Make sure that NEPA_SIGNED_DATE is in datetime, not a character string
df_07_noneg['NEPA_SIGNED_DATE'] = pd.to_datetime(df_07_noneg['NEPA_SIGNED_DATE'])


In [41]:
# There might be something to do here to help with defining project completeness or other 
# ideas having to do with contract awards or initiation.

def df_comm_agg(x):
    names = {
        'NEPA_SIGNED_DATE': x['NEPA_SIGNED_DATE'].min(),
        'REGION': x['ADMIN_REGION'].mean(),
        'DATE_COMP_MIN':  x['DATE_COMPLETED'].min(),
        'DATE_COMP_MAX':  x['DATE_COMPLETED'].max(),
        'PROJECT_DUR_CURRENT': x['DATE_COMPLETED'].max() - x['DATE_COMPLETED'].min(),
        'DATE_AWARD_MIN':  x['DATE_AWARDED'].min(),
        'DATE_AWARD_MAX':  x['DATE_AWARDED'].max(),
        'FY_AWARD_MIN':  x['FISCAL_YEAR_AWARDED'].min(),
        'FY_AWARD_MAX':  x['FISCAL_YEAR_AWARDED'].max(),
        'FY_PLAN_MIN':  x['FISCAL_YEAR_PLANNED'].min(),
        'FY_PLAN_MAX':  x['FISCAL_YEAR_PLANNED'].max(),
        'FY_COMP_MIN':  x['FISCAL_YEAR_COMPLETED'].min(),
        'FY_COMP_MAX':  x['FISCAL_YEAR_COMPLETED'].max(),
        'PROJECT_DUR_PLAN_YR': x['FISCAL_YEAR_PLANNED'].max() - x['FISCAL_YEAR_PLANNED'].min(),
        'ACT_AREA_DIFF': x['ACT_AREA_DIFF'].mean(),
        'TOT_AREA_PLAN': x['NBR_UNITS_PLANNED'].sum(),
        'TOT_AREA_COMP': x['NBR_UNITS_ACCOMPLISHED'].sum(),
        'NBR_ACTIVITIES': x['ACTIVITY_CODE'].count(),
        'FIRE': x['FIRE'].sum(),
        'FIRE_AVE_AREA': x['NBR_UNITS_PLANNED'].loc[x['FIRE'] == 1].mean(),
        'RANGE': x['RANGE'].sum(),
        'RANGE_AVE_AREA': x['NBR_UNITS_PLANNED'].loc[x['RANGE'] == 1].mean(),
        'REC': x['REC'].sum(),
        'REC_AVE_AREA': x['NBR_UNITS_PLANNED'].loc[x['REC'] == 1].mean(),
        'TIMBER': x['TIMBER'].sum(),
        'TIMBER_AVE_AREA': x['NBR_UNITS_PLANNED'].loc[x['TIMBER'] == 1].mean(),
        'SAW': x['SAW'].sum(),
        'SAW_AVE_AREA': x['NBR_UNITS_PLANNED'].loc[x['SAW'] == 1].mean(),
        'WILDLIFE': x['WILDLIFE'].sum(),
        'WILDLIFE_AVE_AREA': x['NBR_UNITS_PLANNED'].loc[x['WILDLIFE'] == 1].mean(),
        'RESTORE': x['RESTORE'].sum(),
        'RESTORE_AVE_AREA': x['NBR_UNITS_PLANNED'].loc[x['RESTORE'] == 1].mean(),
        'MISC': x['MISC'].sum(),
        'MISC_AVE_AREA': x['NBR_UNITS_PLANNED'].loc[x['MISC'] == 1].mean(),
        'ENGI': x['ENGI'].sum(),
        'ENGI_AVE_AREA': x['NBR_UNITS_PLANNED'].loc[x['ENGI'] == 1].mean(),
        'RICHNESS_TEST': x['RICHNESS'].sum(),
        'PERCENT_PROJ_COMP': x['ACT_COMPLETED'].mean(),
        'PLAN_TO_COMP_AVE': x['PLAN_TO_COMPLETE'].mean(),
        'PLAN_TO_AWARD_AVE': x['PLAN_TO_AWARD'].mean(),
        'FIRST_ACTIVITY': x['ACTIVITY'].iat[0],
        'LAST_ACTIVITY': x['ACTIVITY'].iat[-1]}
    

    return pd.Series(names, index=['NEPA_SIGNED_DATE', 'REGION','DATE_COMP_MIN', 'DATE_COMP_MAX', 'PROJECT_DUR_CURRENT',
                                   'PERCENT_PROJ_COMP', 'DATE_AWARD_MIN', 'DATE_AWARD_MAX', 'FY_AWARD_MIN', 'FY_AWARD_MAX',
                                   'FY_PLAN_MIN', 'FY_PLAN_MAX', 
                                   'FY_COMP_MIN', 'FY_COMP_MAX', 'PROJECT_DUR_PLAN_YR', 'ACT_AREA_DIFF', 'TOT_AREA_PLAN', 
                                   'TOT_AREA_COMP', 'NBR_ACTIVITIES', 'FIRE', 'RANGE', 'REC', 'TIMBER', 'SAW', 'WILDLIFE', 
                                   'RESTORE', 'MISC', 'ENGI', 'RICHNESS_TEST', 'PLAN_TO_AWARD_AVE', 'PLAN_TO_COMP_AVE',
                                   'FIRST_ACTIVITY', 'LAST_ACTIVITY',
                                   'FIRE_AVE_AREA', 'RANGE_AVE_AREA', 'REC_AVE_AREA', 'TIMBER_AVE_AREA', 'SAW_AVE_AREA',
                                   'WILDLIFE_AVE_AREA', 'RESTORE_AVE_AREA', 'MISC_AVE_AREA', 'ENGI_AVE_AREA'])




In [42]:
# Apply the df_comm_agg funtion
df_07_agg = df_07_noneg.groupby('NEPA_DOC_NBR').apply(df_comm_agg).reset_index()



In [43]:
# Check the size of the data frame. 
# Should match the number of unique projects from the full dataset
len(df_07_agg)

5419

In [44]:
# Check the new dataframe. Notice that it is sorted by project number
df_07_agg.head(5)

,NEPA_DOC_NBR,NEPA_SIGNED_DATE,REGION,DATE_COMP_MIN,DATE_COMP_MAX,PROJECT_DUR_CURRENT,PERCENT_PROJ_COMP,DATE_AWARD_MIN,DATE_AWARD_MAX,FY_AWARD_MIN,FY_AWARD_MAX,FY_PLAN_MIN,FY_PLAN_MAX,FY_COMP_MIN,FY_COMP_MAX,PROJECT_DUR_PLAN_YR,ACT_AREA_DIFF,TOT_AREA_PLAN,TOT_AREA_COMP,NBR_ACTIVITIES,FIRE,RANGE,REC,TIMBER,SAW,WILDLIFE,RESTORE,MISC,ENGI,RICHNESS_TEST,PLAN_TO_AWARD_AVE,PLAN_TO_COMP_AVE,FIRST_ACTIVITY,LAST_ACTIVITY,FIRE_AVE_AREA,RANGE_AVE_AREA,REC_AVE_AREA,TIMBER_AVE_AREA,SAW_AVE_AREA,WILDLIFE_AVE_AREA,RESTORE_AVE_AREA,MISC_AVE_AREA,ENGI_AVE_AREA
0,10031,2009-07-17,8.0,2011-07-15,2019-04-23,2839 days,0.608696,2010-08-15,2019-04-23,2010.0,2019.0,2010,2016,2011.0,2019.0,6,-8.244444,2623.0,2216.0,46,4,0,0,0,0,38,4,0,0,0,0.111111,1.678571,Wildlife Habitat Regeneration cut,Wildlife Habitat Mechanical treatment,432.250000,NaN,NaN,NaN,NaN,23.342105,1.75,NaN,NaN
1,10045,2012-01-17,8.0,2013-08-13,2019-09-30,2239 days,1.000000,2013-08-13,2019-09-30,2013.0,2019.0,2013,2019,2013.0,2019.0,6,-0.041304,1168.9,1167.0,46,0,6,1,39,0,0,0,0,0,0,0.260870,0.260870,Tree Release and Weed,Sign Maintenance,NaN,50.816667,9.0,21.923077,NaN,NaN,NaN,NaN,NaN
2,1006,2009-10-16,5.0,2010-08-09,2021-11-10,4111 days,0.305882,2010-08-02,2021-11-10,2010.0,2022.0,2010,2023,2010.0,2022.0,13,-0.981132,6630.0,1858.0,170,83,7,0,80,0,0,0,0,0,0,0.566038,0.673077,Precommercial Thin,Rearrangement of Fuels,17.096386,219.285714,NaN,45.950000,NaN,NaN,NaN,NaN,NaN
3,10083,2011-07-01,5.0,2011-09-09,2018-01-09,2314 days,0.867647,2011-09-06,2017-10-01,2011.0,2018.0,2011,2019,2011.0,2018.0,8,0.022222,2489.2,2265.0,68,51,8,0,9,0,0,0,0,0,0,0.333333,1.033898,Burning of Piled Material,Commercial Thin,36.494118,0.987500,NaN,68.900000,NaN,NaN,NaN,NaN,NaN
4,10130,2009-01-15,6.0,2009-11-19,2019-07-31,3541 days,0.863372,2009-09-09,2018-12-16,2009.0,2019.0,2009,2026,2010.0,2019.0,17,0.003300,13990.1,12309.1,344,140,3,0,146,55,0,0,0,0,0,0.168317,1.141414,Burning of Piled Material,Burning of Piled Material,18.165000,155.333333,NaN,71.815068,9.018182,NaN,NaN,NaN,NaN


In [45]:
# Check that the diversity metric dataframe is sorted by project number
df_div_mets.head(10)

,index,NEPA_DOC_NBR,PROJ_RICHNESS,PROJ_DIVERSE,PROJ_EVENNESS,AREA_DOC_NBR,AREA_RICHNESS,AREA_DIVERSE,AREA_EVENNESS
0,4554,10031,3,0.582584,0.530291,10031,1,-0.000000,NaN
1,3888,10045,3,0.488871,0.444989,10045,7,0.620920,0.319090
2,2294,1006,3,0.836109,0.761059,1006,74,4.101769,0.952999
3,2131,10083,3,0.735189,0.669198,10083,34,3.388206,0.960822
4,3867,10130,4,1.064084,0.767575,10130,134,4.755605,0.970960
5,1398,10212,3,0.799846,0.728052,10212,74,4.232771,0.983436
6,962,10348,3,0.596580,0.543030,10348,2,0.081510,0.117595
7,1814,10383,1,-0.000000,NaN,10383,1,-0.000000,NaN
8,489,10457,4,0.544810,0.392997,10457,18,2.859266,0.989238
9,1244,10501,1,-0.000000,NaN,10501,9,2.163956,0.984859


In [46]:
# Add the richness, diversity, and evenness values to the dataframe 

df_07_agg['PROJ_RICHNESS'] = df_div_mets['PROJ_RICHNESS']
df_07_agg['PROJ_DIVERSE'] = df_div_mets['PROJ_DIVERSE']
df_07_agg['PROJ_EVENNESS'] = df_div_mets['PROJ_EVENNESS']

df_07_agg['AREA_RICHNESS'] = df_div_mets['AREA_RICHNESS']
df_07_agg['AREA_DIVERSE'] = df_div_mets['AREA_DIVERSE']
df_07_agg['AREA_EVENNESS'] = df_div_mets['AREA_EVENNESS']


In [47]:
# Check the updated data frame
df_07_agg.head(10)

,NEPA_DOC_NBR,NEPA_SIGNED_DATE,REGION,DATE_COMP_MIN,DATE_COMP_MAX,PROJECT_DUR_CURRENT,PERCENT_PROJ_COMP,DATE_AWARD_MIN,DATE_AWARD_MAX,FY_AWARD_MIN,FY_AWARD_MAX,FY_PLAN_MIN,FY_PLAN_MAX,FY_COMP_MIN,FY_COMP_MAX,PROJECT_DUR_PLAN_YR,ACT_AREA_DIFF,TOT_AREA_PLAN,TOT_AREA_COMP,NBR_ACTIVITIES,FIRE,RANGE,REC,TIMBER,SAW,WILDLIFE,RESTORE,MISC,ENGI,RICHNESS_TEST,PLAN_TO_AWARD_AVE,PLAN_TO_COMP_AVE,FIRST_ACTIVITY,LAST_ACTIVITY,FIRE_AVE_AREA,RANGE_AVE_AREA,REC_AVE_AREA,TIMBER_AVE_AREA,SAW_AVE_AREA,WILDLIFE_AVE_AREA,RESTORE_AVE_AREA,MISC_AVE_AREA,ENGI_AVE_AREA,PROJ_RICHNESS,PROJ_DIVERSE,PROJ_EVENNESS,AREA_RICHNESS,AREA_DIVERSE,AREA_EVENNESS
0,10031,2009-07-17,8.0,2011-07-15,2019-04-23,2839 days,0.608696,2010-08-15,2019-04-23,2010.0,2019.0,2010,2016,2011.0,2019.0,6,-8.244444,2623.0,2216.0,46,4,0,0,0,0,38,4,0,0,0,0.111111,1.678571,Wildlife Habitat Regeneration cut,Wildlife Habitat Mechanical treatment,432.250000,NaN,NaN,NaN,NaN,23.342105,1.75,NaN,NaN,3,0.582584,0.530291,1,-0.000000,NaN
1,10045,2012-01-17,8.0,2013-08-13,2019-09-30,2239 days,1.000000,2013-08-13,2019-09-30,2013.0,2019.0,2013,2019,2013.0,2019.0,6,-0.041304,1168.9,1167.0,46,0,6,1,39,0,0,0,0,0,0,0.260870,0.260870,Tree Release and Weed,Sign Maintenance,NaN,50.816667,9.0,21.923077,NaN,NaN,NaN,NaN,NaN,3,0.488871,0.444989,7,0.620920,0.319090
2,1006,2009-10-16,5.0,2010-08-09,2021-11-10,4111 days,0.305882,2010-08-02,2021-11-10,2010.0,2022.0,2010,2023,2010.0,2022.0,13,-0.981132,6630.0,1858.0,170,83,7,0,80,0,0,0,0,0,0,0.566038,0.673077,Precommercial Thin,Rearrangement of Fuels,17.096386,219.285714,NaN,45.950000,NaN,NaN,NaN,NaN,NaN,3,0.836109,0.761059,74,4.101769,0.952999
3,10083,2011-07-01,5.0,2011-09-09,2018-01-09,2314 days,0.867647,2011-09-06,2017-10-01,2011.0,2018.0,2011,2019,2011.0,2018.0,8,0.022222,2489.2,2265.0,68,51,8,0,9,0,0,0,0,0,0,0.333333,1.033898,Burning of Piled Material,Commercial Thin,36.494118,0.987500,NaN,68.900000,NaN,NaN,NaN,NaN,NaN,3,0.735189,0.669198,34,3.388206,0.960822
4,10130,2009-01-15,6.0,2009-11-19,2019-07-31,3541 days,0.863372,2009-09-09,2018-12-16,2009.0,2019.0,2009,2026,2010.0,2019.0,17,0.003300,13990.1,12309.1,344,140,3,0,146,55,0,0,0,0,0,0.168317,1.141414,Burning of Piled Material,Burning of Piled Material,18.165000,155.333333,NaN,71.815068,9.018182,NaN,NaN,NaN,NaN,4,1.064084,0.767575,134,4.755605,0.970960
5,10212,2009-07-24,5.0,2009-08-04,2021-01-23,4190 days,0.905213,2009-08-04,2021-01-23,2009.0,2021.0,2009,2021,2009.0,2021.0,12,0.000000,1405.7,1214.5,211,130,0,0,72,0,0,0,9,0,0,0.381910,0.890052,Burning of Piled Material,Improvement Cut,6.317692,NaN,NaN,7.236111,NaN,NaN,NaN,7.044444,NaN,3,0.799846,0.728052,74,4.232771,0.983436
6,10348,2009-09-29,4.0,2010-10-01,2020-08-15,3606 days,0.599278,2010-10-01,2018-06-01,2011.0,2018.0,2011,2024,2011.0,2020.0,13,0.245763,17082.0,15904.2,277,42,12,0,223,0,0,0,0,0,0,0.055085,1.843373,"Salvage Cut (intermediate treatment, not regen...",Commercial Thin,88.738095,1.166667,NaN,59.825112,NaN,NaN,NaN,NaN,NaN,3,0.596580,0.543030,2,0.081510,0.117595
7,10383,2010-04-06,1.0,NaT,NaT,NaT,0.000000,NaT,NaT,NaN,NaN,2013,2013,NaN,NaN,0,NaN,5.0,0.0,1,0,0,0,1,0,0,0,0,0,0,NaN,NaN,Patch Clearcut (EA/RH/FH),Patch Clearcut (EA/RH/FH),NaN,NaN,NaN,5.000000,NaN,NaN,NaN,NaN,NaN,1,-0.000000,NaN,1,-0.000000,NaN
8,10457,2009-01-14,3.0,2009-06-06,2017-04-22,2877 days,1.000000,2009-06-06,2017-04-22,2009.0,2017.0,2008,2017,2009.0,2017.0,9,0.626087,12899.0,12971.0,115,19,1,0,94,0,1,0,0,0,0,0.356522,0.756522,Burning of Piled Material,Stand Silviculture Prescription,106.894737,131.000000,NaN,112.829787,NaN,131.000000,NaN,NaN,NaN,4,0.544810,0.392997,18,2.859266,0.989238
9,10501,2009-11-18,1.0,2009-12-18,2009-12-21,3 days,1.000000,2009-12-18,2009-12-21,2010.0,2010.0,2007,2010,2010.0,2010.0,3,0.000000,736.0,736.0,10,0,0,0,10,0,0,0,0,0,0,0.300000,0.300000,Stand Silviculture Prescription,Stand Silviculture Prescription,NaN,NaN,NaN,73.600000,NaN,NaN,NaN,NaN,NaN,1,-0.000000,NaN,9,2.163956,0.984859


In [48]:
# Calculate the values to go into the RICHNESS_TEST column
# This will be used as a check against the PROJ_RICHNESS variable
# calculated earlier

richness = []
#tmp_rich = 0

for index, row in df_07_agg.iterrows():
    tmp_rich = 0
    if row['FIRE'] > 0:
        tmp_rich += 1
    if row['RANGE'] > 0:
        tmp_rich += 1
    if row['REC'] > 0:
        tmp_rich += 1
    if row['TIMBER'] > 0:
        tmp_rich += 1
    if row['SAW'] > 0:
        tmp_rich += 1
    if row['WILDLIFE'] > 0:
        tmp_rich += 1   
    if row['RESTORE'] > 0:
        tmp_rich += 1
    if row['MISC'] > 0:
        tmp_rich += 1    
    if row['ENGI'] > 0:
        tmp_rich += 1
    richness.append(tmp_rich)
    
len(richness)

5419

In [49]:
df_07_agg['RICHNESS_TEST'] = np.array(richness)

In [50]:
df_07_agg.head(10)

,NEPA_DOC_NBR,NEPA_SIGNED_DATE,REGION,DATE_COMP_MIN,DATE_COMP_MAX,PROJECT_DUR_CURRENT,PERCENT_PROJ_COMP,DATE_AWARD_MIN,DATE_AWARD_MAX,FY_AWARD_MIN,FY_AWARD_MAX,FY_PLAN_MIN,FY_PLAN_MAX,FY_COMP_MIN,FY_COMP_MAX,PROJECT_DUR_PLAN_YR,ACT_AREA_DIFF,TOT_AREA_PLAN,TOT_AREA_COMP,NBR_ACTIVITIES,FIRE,RANGE,REC,TIMBER,SAW,WILDLIFE,RESTORE,MISC,ENGI,RICHNESS_TEST,PLAN_TO_AWARD_AVE,PLAN_TO_COMP_AVE,FIRST_ACTIVITY,LAST_ACTIVITY,FIRE_AVE_AREA,RANGE_AVE_AREA,REC_AVE_AREA,TIMBER_AVE_AREA,SAW_AVE_AREA,WILDLIFE_AVE_AREA,RESTORE_AVE_AREA,MISC_AVE_AREA,ENGI_AVE_AREA,PROJ_RICHNESS,PROJ_DIVERSE,PROJ_EVENNESS,AREA_RICHNESS,AREA_DIVERSE,AREA_EVENNESS
0,10031,2009-07-17,8.0,2011-07-15,2019-04-23,2839 days,0.608696,2010-08-15,2019-04-23,2010.0,2019.0,2010,2016,2011.0,2019.0,6,-8.244444,2623.0,2216.0,46,4,0,0,0,0,38,4,0,0,3,0.111111,1.678571,Wildlife Habitat Regeneration cut,Wildlife Habitat Mechanical treatment,432.250000,NaN,NaN,NaN,NaN,23.342105,1.75,NaN,NaN,3,0.582584,0.530291,1,-0.000000,NaN
1,10045,2012-01-17,8.0,2013-08-13,2019-09-30,2239 days,1.000000,2013-08-13,2019-09-30,2013.0,2019.0,2013,2019,2013.0,2019.0,6,-0.041304,1168.9,1167.0,46,0,6,1,39,0,0,0,0,0,3,0.260870,0.260870,Tree Release and Weed,Sign Maintenance,NaN,50.816667,9.0,21.923077,NaN,NaN,NaN,NaN,NaN,3,0.488871,0.444989,7,0.620920,0.319090
2,1006,2009-10-16,5.0,2010-08-09,2021-11-10,4111 days,0.305882,2010-08-02,2021-11-10,2010.0,2022.0,2010,2023,2010.0,2022.0,13,-0.981132,6630.0,1858.0,170,83,7,0,80,0,0,0,0,0,3,0.566038,0.673077,Precommercial Thin,Rearrangement of Fuels,17.096386,219.285714,NaN,45.950000,NaN,NaN,NaN,NaN,NaN,3,0.836109,0.761059,74,4.101769,0.952999
3,10083,2011-07-01,5.0,2011-09-09,2018-01-09,2314 days,0.867647,2011-09-06,2017-10-01,2011.0,2018.0,2011,2019,2011.0,2018.0,8,0.022222,2489.2,2265.0,68,51,8,0,9,0,0,0,0,0,3,0.333333,1.033898,Burning of Piled Material,Commercial Thin,36.494118,0.987500,NaN,68.900000,NaN,NaN,NaN,NaN,NaN,3,0.735189,0.669198,34,3.388206,0.960822
4,10130,2009-01-15,6.0,2009-11-19,2019-07-31,3541 days,0.863372,2009-09-09,2018-12-16,2009.0,2019.0,2009,2026,2010.0,2019.0,17,0.003300,13990.1,12309.1,344,140,3,0,146,55,0,0,0,0,4,0.168317,1.141414,Burning of Piled Material,Burning of Piled Material,18.165000,155.333333,NaN,71.815068,9.018182,NaN,NaN,NaN,NaN,4,1.064084,0.767575,134,4.755605,0.970960
5,10212,2009-07-24,5.0,2009-08-04,2021-01-23,4190 days,0.905213,2009-08-04,2021-01-23,2009.0,2021.0,2009,2021,2009.0,2021.0,12,0.000000,1405.7,1214.5,211,130,0,0,72,0,0,0,9,0,3,0.381910,0.890052,Burning of Piled Material,Improvement Cut,6.317692,NaN,NaN,7.236111,NaN,NaN,NaN,7.044444,NaN,3,0.799846,0.728052,74,4.232771,0.983436
6,10348,2009-09-29,4.0,2010-10-01,2020-08-15,3606 days,0.599278,2010-10-01,2018-06-01,2011.0,2018.0,2011,2024,2011.0,2020.0,13,0.245763,17082.0,15904.2,277,42,12,0,223,0,0,0,0,0,3,0.055085,1.843373,"Salvage Cut (intermediate treatment, not regen...",Commercial Thin,88.738095,1.166667,NaN,59.825112,NaN,NaN,NaN,NaN,NaN,3,0.596580,0.543030,2,0.081510,0.117595
7,10383,2010-04-06,1.0,NaT,NaT,NaT,0.000000,NaT,NaT,NaN,NaN,2013,2013,NaN,NaN,0,NaN,5.0,0.0,1,0,0,0,1,0,0,0,0,0,1,NaN,NaN,Patch Clearcut (EA/RH/FH),Patch Clearcut (EA/RH/FH),NaN,NaN,NaN,5.000000,NaN,NaN,NaN,NaN,NaN,1,-0.000000,NaN,1,-0.000000,NaN
8,10457,2009-01-14,3.0,2009-06-06,2017-04-22,2877 days,1.000000,2009-06-06,2017-04-22,2009.0,2017.0,2008,2017,2009.0,2017.0,9,0.626087,12899.0,12971.0,115,19,1,0,94,0,1,0,0,0,4,0.356522,0.756522,Burning of Piled Material,Stand Silviculture Prescription,106.894737,131.000000,NaN,112.829787,NaN,131.000000,NaN,NaN,NaN,4,0.544810,0.392997,18,2.859266,0.989238
9,10501,2009-11-18,1.0,2009-12-18,2009-12-21,3 days,1.000000,2009-12-18,2009-12-21,2010.0,2010.0,2007,2010,2010.0,2010.0,3,0.000000,736.0,736.0,10,0,0,0,10,0,0,0,0,0,1,0.300000,0.300000,Stand Silviculture Prescription,Stand Silviculture Prescription,NaN,NaN,NaN,73.600000,NaN,NaN,NaN,NaN,NaN,1,-0.000000,NaN,9,2.163956,0.984859


In [51]:
# Convert all of the time delta datatypes to numeric (integers).

df_07_agg['PROJECT_DUR_CURRENT'] = pd.to_numeric(df_07_agg['PROJECT_DUR_CURRENT'].dt.days, downcast='integer')


In [52]:
# Double check the datatypes. Make sure there are no time delta data types. 
# Time deltas can be  difficult to work with from a .csv file. 

df_07_agg.dtypes

NEPA_DOC_NBR                   object
NEPA_SIGNED_DATE       datetime64[ns]
REGION                        float64
DATE_COMP_MIN          datetime64[ns]
DATE_COMP_MAX          datetime64[ns]
PROJECT_DUR_CURRENT           float64
PERCENT_PROJ_COMP             float64
DATE_AWARD_MIN         datetime64[ns]
DATE_AWARD_MAX         datetime64[ns]
FY_AWARD_MIN                  float64
FY_AWARD_MAX                  float64
FY_PLAN_MIN                     int64
FY_PLAN_MAX                     int64
FY_COMP_MIN                   float64
FY_COMP_MAX                   float64
PROJECT_DUR_PLAN_YR             int64
ACT_AREA_DIFF                 float64
TOT_AREA_PLAN                 float64
TOT_AREA_COMP                 float64
NBR_ACTIVITIES                  int64
FIRE                            int64
RANGE                           int64
REC                             int64
TIMBER                          int64
SAW                             int64
WILDLIFE                        int64
RESTORE     

In [54]:
# Optional: save to csv 
# This is so that the FACTS can be more easliy merged
# with any updated PALS

#df_07_agg.to_csv("", index=False)


## 6. Combine with the PALS Data

In [56]:
# Optional: Read in the .csv file created in the code block above 
#df_07_agg = pd.read_csv(".csv")
df_07_agg.head(2)


,NEPA_DOC_NBR,NEPA_SIGNED_DATE,REGION,DATE_COMP_MIN,DATE_COMP_MAX,PROJECT_DUR_CURRENT,PERCENT_PROJ_COMP,DATE_AWARD_MIN,DATE_AWARD_MAX,FY_AWARD_MIN,FY_AWARD_MAX,FY_PLAN_MIN,FY_PLAN_MAX,FY_COMP_MIN,FY_COMP_MAX,PROJECT_DUR_PLAN_YR,ACT_AREA_DIFF,TOT_AREA_PLAN,TOT_AREA_COMP,NBR_ACTIVITIES,FIRE,RANGE,REC,TIMBER,SAW,WILDLIFE,RESTORE,MISC,ENGI,RICHNESS_TEST,PLAN_TO_AWARD_AVE,PLAN_TO_COMP_AVE,FIRST_ACTIVITY,LAST_ACTIVITY,FIRE_AVE_AREA,RANGE_AVE_AREA,REC_AVE_AREA,TIMBER_AVE_AREA,SAW_AVE_AREA,WILDLIFE_AVE_AREA,RESTORE_AVE_AREA,MISC_AVE_AREA,ENGI_AVE_AREA,PROJ_RICHNESS,PROJ_DIVERSE,PROJ_EVENNESS,AREA_RICHNESS,AREA_DIVERSE,AREA_EVENNESS
0,10031,2009-07-17,8.0,2011-07-15,2019-04-23,2839.0,0.608696,2010-08-15,2019-04-23,2010.0,2019.0,2010,2016,2011.0,2019.0,6,-8.244444,2623.0,2216.0,46,4,0,0,0,0,38,4,0,0,3,0.111111,1.678571,Wildlife Habitat Regeneration cut,Wildlife Habitat Mechanical treatment,432.25,NaN,NaN,NaN,NaN,23.342105,1.75,NaN,NaN,3,0.582584,0.530291,1,-0.00000,NaN
1,10045,2012-01-17,8.0,2013-08-13,2019-09-30,2239.0,1.000000,2013-08-13,2019-09-30,2013.0,2019.0,2013,2019,2013.0,2019.0,6,-0.041304,1168.9,1167.0,46,0,6,1,39,0,0,0,0,0,3,0.260870,0.260870,Tree Release and Weed,Sign Maintenance,NaN,50.816667,9.0,21.923077,NaN,NaN,NaN,NaN,NaN,3,0.488871,0.444989,7,0.62092,0.31909


In [57]:
len(df_07_agg)

5419

In [60]:
# Optional: if df_07_agg may need to reconvert the dates to datetime.
df_07_agg['NEPA_SIGNED_DATE'] = pd.to_datetime(df_07_agg['NEPA_SIGNED_DATE'])
df_07_agg['DATE_COMP_MIN'] = pd.to_datetime(df_07_agg['DATE_COMP_MIN'])
df_07_agg['DATE_COMP_MAX'] = pd.to_datetime(df_07_agg['DATE_COMP_MAX'])
df_07_agg['DATE_AWARD_MIN'] = pd.to_datetime(df_07_agg['DATE_AWARD_MIN'])
df_07_agg['DATE_AWARD_MAX'] = pd.to_datetime(df_07_agg['DATE_AWARD_MAX'])


In [62]:
# Double check the data types
df_07_agg.dtypes

NEPA_DOC_NBR                   object
NEPA_SIGNED_DATE       datetime64[ns]
REGION                        float64
DATE_COMP_MIN          datetime64[ns]
DATE_COMP_MAX          datetime64[ns]
PROJECT_DUR_CURRENT           float64
PERCENT_PROJ_COMP             float64
DATE_AWARD_MIN         datetime64[ns]
DATE_AWARD_MAX         datetime64[ns]
FY_AWARD_MIN                  float64
FY_AWARD_MAX                  float64
FY_PLAN_MIN                     int64
FY_PLAN_MAX                     int64
FY_COMP_MIN                   float64
FY_COMP_MAX                   float64
PROJECT_DUR_PLAN_YR             int64
ACT_AREA_DIFF                 float64
TOT_AREA_PLAN                 float64
TOT_AREA_COMP                 float64
NBR_ACTIVITIES                  int64
FIRE                            int64
RANGE                           int64
REC                             int64
TIMBER                          int64
SAW                             int64
WILDLIFE                        int64
RESTORE     

**NOTE!!!**

To create the csv that will be **df** in the figures notebook use **df_pals**. 
This is the "raw" PALS data, not merged with the Shannon Index data. 


To create the csv that will be **df_foot** for the figures notebook use **df_na**. 
This is the PALS data merged with the Shannon Index data in section 2.

In [ ]:
df_pals.head(5)

In [63]:
df_na.head(5)

,FOREST_ID,NEPA_DOC_NBR,PROJECT NAME,LMU (ACTUAL),LMU – REGION,LMU – FOREST,LMU – DISTRICT,PROJECT STATUS,PROJECT CREATED,CREATED FY,DECISION ID,DECISION NAME,INITIATION DATE,INITIATION FY,DECISION SIGNED,SIGNED FY,SIGNER LAST NAME,SIGNER FIRST NAME,SIGNER TITLE,DECISION TYPE,DECISION APPEAL RULE,PROJECT NOTICE AND COMMENT REGULATION,APPEALED OR OBJECTED?,NO COMMENTS OR ONLY SUPPORT?,LITIGATED?,FC Facility management – purpose,FR Research – purpose,HF Fuels management – purpose,HR Heritage resource management – purpose,LM Land ownership management – purpose,LW Land acquisition – purpose,MG Minerals and geology – purpose,PN Land management planning – purpose,RD Road management – purpose,RG Grazing management – purpose,"RO Regulations, directives, orders – purpose",RU Special area management – purpose,RW Recreation management – purpose,SU Special use management – purpose,TM Forest products – purpose,VM Vegetation management (non-forest products) – purpose,"WF Wildlife, fish, rare plants – purpose",WM Water management – purpose,AL Land use adjustments – activity,BL Boundary adjustments – activity,BM Biomass – activity,CP Plan creation/revision – activity,DC Directive creation/modification – activity,DR Road decommissioning – activity,DS Developed site management – activity,EC Environmental compliance actions – activity,ET Electric transmission – activity,"FACTS Activity (0,1)",FI Facility improvements/construction – activity,FN Fuel treatments – activity,FV Forest vegetation improvements – activity,GA Dispersed recreation management – activity,GP Grazing allotment management – activity,GR Grazing authorizations – activity,GT Geothermal – activity,HI Species habitat improvements – activity,HP Hydropower – activity,HR Heritage resource management – activity,LA Special use authorizations – activity,LP Land purchases – activity,MF Facility maintenance – activity,ML Abandoned mine land clean-up – activity,MO Minerals or geology plans of operations – activity,MP Plan amendment – activity,MT Trail management – activity,NC Special products sales – activity,NG Natural gas – activity,NW Noxious weed treatments – activity,OC Order creation/modification – activity,OL Oil – activity,PE Species population enhancements – activity,PJ Land exchanges – activity,RA Roadless area management – activity,RC Regulation creation/modification – activity,RD Road maintenance – activity,RE Research and development – activity,RI Road improvements/construction – activity,RV Rangeland vegetation improvements – activity,SA Special area management – activity,SC Scenery management – activity,SI Grazing structural improvements – activity,SL Solar – activity,SS Timber sales (salvage) – activity,TR Travel management – activity,TS Timber sales (green) – activity,WC Watershed improvements – activity,WD Wilderness management – activity,WI Wind – activity,UNIQUE PROJECT?,ELAPSED DAYS,UNIQUE DECISION?,ongoing,DISTRICT_ID,REGION_ID,DECISION_LEVEL,RegionName,forest,calendarYearSigned,calendarYearInitiated,numact,H,projarea,footprint,E
13,102.0,47722,Supplemental EIS - BDNF Land & Resource Manage...,Beaverhead-Deerlodge National Forest All Units...,R1,BDNF,Beaverhead-Deerlodge National Forest All Units...,Complete,09/14/2015,2015.0,46316.0,Supplemental EIS - BDNF Land & Resource Manage...,2015-09-11,2015.0,2016-10-11,2017.0,Glossa,Melany,Forest Supervisor,ROD,NaN,219 (2012),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,396.0,NaN,NaN,10200.0,1.0,National Forest/Grassland,Northern,Beaverhead-Deerlodge,2016.0,2015.0,1.0,0.000000,2.599100e+07,1.552920e+07,0.000000
20,102.0,42302,Forest Plan Temporary Roads,Beaverhead-Deerlodge National Forest All Units...,R1,BDNF,Beaverhead-Deerlodge National Forest All Units...,Complete,09/11/2014,2014.0,42

In [64]:
print(df_na['INITIATION DATE'].min())
print(df_na['INITIATION DATE'].max())

1998-10-23 00:00:00
2021-06-25 00:00:00


In [66]:
# Check for leading and trailing 0s

facts_proj = list(df_07_agg['NEPA_DOC_NBR'].unique())
pals_proj = list(df_na['NEPA_DOC_NBR'].unique())

In [67]:
print(len(facts_proj))
print(len(pals_proj))

5419
4823


In [68]:
facts_lead0_rem = [s.lstrip("0") for s in facts_proj]
pals_lead0_rem = [s.lstrip("0") for s in pals_proj]


In [69]:
print(len(facts_lead0_rem))
print(len(pals_lead0_rem))

5419
4823


## 7. Merge the datasets, then filter by NEPA initiation dates between 2009-2021 and keep all project purposes

Repeat sections 7-9 twice. Once merging df_pals and df_07_agg. Make sure to change the name of the csv file you are saving in section 9.

In [70]:
#df_pals_comm = pd.merge(df_pals, df_07_agg, how="left", on='NEPA_DOC_NBR')
df_pals_comm = pd.merge(df_na, df_07_agg, how="left", on='NEPA_DOC_NBR')


In [71]:
# The total dataset should be the same length as the PALS-Shannon dataset
len(df_pals_comm)


4823

In [72]:
# Filter by NEPA initiaion >=2009-01-01 and <=2021-12-31
# and create a new data frame

mask = (df_pals_comm['INITIATION DATE'] >= '2009-01-01') & (df_pals_comm['INITIATION DATE'] <= '2021-12-31')

df_pals_comm_09 = df_pals_comm.loc[mask]


In [73]:
# Check the length of the new data frame
# Check the number of projects by NEPA type

print(len(df_pals_comm_09))
df_pals_comm_09['DECISION TYPE'].value_counts()

4196


DM     2509
DN     1512
ROD     175
Name: DECISION TYPE, dtype: int64

## 8. Now that we have merged the data sets and filtered by the NEPA Initiation date there are a few more variables to calculate

* **TOTAL_PROJECT_DAYS** - the time from NEPA initiation to the last completed activity in a project
* **TOTAL_PROJECT_PLAN_FY** - the time from NEPA initiation (by fiscal year) to when the last activity is planned to be completed (by fiscal year)
* **NEPA_TO_LAST_ACT** - the time from NEPA signed to the last activity in a project completed
* **NEPA_TO_FIRST_ACT** - the time from NEPA signed to the first activity in a project completed
* **NEPA_TO_CONTRACT** - the time from NEPA signed to the first contract of the project awarded 

Make sure to convert all dates to datetimes and all time deltas to integers.

In [74]:
# Calculate a few other variables: 

df_pals_comm_09['INITIATION DATE'] = pd.to_datetime(df_pals_comm_09['INITIATION DATE'])
df_pals_comm_09['DECISION SIGNED'] = pd.to_datetime(df_pals_comm_09['DECISION SIGNED'])

df_pals_comm_09['TOTAL_PROJECT_DAYS'] = df_pals_comm_09['DATE_COMP_MAX'] - df_pals_comm_09['INITIATION DATE']
df_pals_comm_09['TOTAL_PROJECT_PLAN_FY'] = df_pals_comm_09['FY_PLAN_MAX'] - df_pals_comm_09['INITIATION FY']
df_pals_comm_09['NEPA_TO_LAST_ACT'] = df_pals_comm_09['DATE_COMP_MAX'] - df_pals_comm_09['DECISION SIGNED']
df_pals_comm_09['NEPA_TO_FIRST_ACT'] = df_pals_comm_09['DATE_COMP_MIN'] - df_pals_comm_09['DECISION SIGNED']
df_pals_comm_09['NEPA_TO_CONTRACT'] = df_pals_comm_09['DATE_AWARD_MIN'] - df_pals_comm_09['DECISION SIGNED']

df_pals_comm_09['TOTAL_PROJECT_DAYS'] = pd.to_numeric(df_pals_comm_09['TOTAL_PROJECT_DAYS'].dt.days, downcast='integer')
df_pals_comm_09['NEPA_TO_LAST_ACT'] = pd.to_numeric(df_pals_comm_09['NEPA_TO_LAST_ACT'].dt.days, downcast='integer')
df_pals_comm_09['NEPA_TO_FIRST_ACT'] = pd.to_numeric(df_pals_comm_09['NEPA_TO_FIRST_ACT'].dt.days, downcast='integer')
df_pals_comm_09['NEPA_TO_CONTRACT'] = pd.to_numeric(df_pals_comm_09['NEPA_TO_CONTRACT'].dt.days, downcast='integer')



/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/109884008.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pals_comm_09['INITIATION DATE'] = pd.to_datetime(df_pals_comm_09['INITIATION DATE'])
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/109884008.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pals_comm_09['DECISION SIGNED'] = pd.to_datetime(df_pals_comm_09['DECISION SIGNED'])
/var/folders/rr/nq1j46pd355fn_0gk__qf1s40000gn/T/ipykernel_30304/109884008.py:6: S

In [77]:
# Double check data types.
# We don't want any timedelta to be saved to the csv!!!

df_pals_comm_09.dtypes

FOREST_ID                                                          float64
NEPA_DOC_NBR                                                        object
PROJECT NAME                                                        object
LMU (ACTUAL)                                                        object
LMU – REGION                                                        object
LMU – FOREST                                                        object
LMU – DISTRICT                                                      object
PROJECT STATUS                                                      object
PROJECT CREATED                                                     object
CREATED FY                                                         float64
DECISION ID                                                        float64
DECISION NAME                                                       object
INITIATION DATE                                             datetime64[ns]
INITIATION FY            

In [78]:
# Check the NEPA initiation dates (these are from PALS)

print(df_pals_comm_09['INITIATION DATE'].min())
print(df_pals_comm_09['INITIATION DATE'].max())

2009-01-01 00:00:00
2021-06-25 00:00:00


In [79]:
# Check the NEPA_SIGNED_DATES (these are from FACTS)

print(df_pals_comm_09['NEPA_SIGNED_DATE'].min())
print(df_pals_comm_09['NEPA_SIGNED_DATE'].max())

2009-03-16 00:00:00
2021-09-09 00:00:00


In [80]:
# Check the NEPA signed dates (DECISION SIGNED, from PALS)

print(df_pals_comm_09['DECISION SIGNED'].min())
print(df_pals_comm_09['DECISION SIGNED'].max())

2009-01-19 00:00:00
2021-09-15 00:00:00


In [81]:
# Check the proportion of NEPA types

df_pals_comm_09['DECISION TYPE'].value_counts()

DM     2509
DN     1512
ROD     175
Name: DECISION TYPE, dtype: int64

In [82]:
# Drop some of the columns. 
# Mostly the activity purposes from the PALS dataset

df_pals_comm_09_sub = df_pals_comm_09.drop(['AL Land use adjustments – activity', 'BL Boundary adjustments – activity',
                                                     'BM Biomass – activity', 'CP Plan creation/revision – activity',
                                                     'DC Directive creation/modification – activity', 'DR Road decommissioning – activity',
                                                     'DS Developed site management – activity', 'EC Environmental compliance actions – activity',
                                                     'ET Electric transmission – activity', 'FI Facility improvements/construction – activity',
                                                     'FN Fuel treatments – activity', 'FV Forest vegetation improvements – activity',
                                                     'GA Dispersed recreation management – activity', 'GP Grazing allotment management – activity',
                                                     'GR Grazing authorizations – activity', 'GT Geothermal – activity', 'HI Species habitat improvements – activity',
                                                     'HP Hydropower – activity', 'HR Heritage resource management – activity',
                                                     'LA Special use authorizations – activity', 'LP Land purchases – activity',
                                                     'MF Facility maintenance – activity', 'ML Abandoned mine land clean-up – activity',
                                                     'MO Minerals or geology plans of operations – activity', 'MP Plan amendment – activity',
                                                     'MT Trail management – activity', 'NC Special products sales – activity', 'NG Natural gas – activity',
                                                     'NW Noxious weed treatments – activity', 'OC Order creation/modification – activity', 'OL Oil – activity',
                                                     'PE Species population enhancements – activity', 'PJ Land exchanges – activity',
                                                     'RA Roadless area management – activity', 'RC Regulation creation/modification – activity', 
                                                     'RD Road maintenance – activity', 'RE Research and development – activity', 'RI Road improvements/construction – activity',
                                                     'RV Rangeland vegetation improvements – activity', 'SA Special area management – activity', 
                                                     'SC Scenery management – activity', 'SI Grazing structural improvements – activity', 
                                                     'SI Grazing structural improvements – activity', 'SL Solar – activity', 'SS Timber sales (salvage) – activity',
                                                     'TR Travel management – activity', 'TS Timber sales (green) – activity',
                                                     'WC Watershed improvements – activity', 'WD Wilderness management – activity',
                                                     'WI Wind – activity'], axis=1)




## 9. Save to a .csv file

### **!!STOP!!** 
Make sure to change the name of the csv file to reflect whether the FACTS data (df_07_agg) was merged with the "raw" PALS (df_pals) or the PALS with the Shannon Index data (df_na)

In [83]:
# Save to csv

## FACTS+PALS: NEPA Init 2009-2021, all purposes, PALS 2022 dataset, calculated project footprint
#df_pals_comm_09.to_csv(raw_data_dir + "df_pals_comm_NEPA_init_2009_2021_allpurp_footprint_c20230123.csv", index=False)
#df_pals_comm_09_sub.to_csv(raw_data_dir + "df_pals_comm_NEPA_init_2009_2021_allpurp_footprint_sub_c202301232.csv", index=False)

